### **Parte 1:** Importanción de librerías

#### Suprimiendo los Warnings

In [1]:
import warnings
warnings.filterwarnings("ignore")

#### Importando las librerías para trabajar CNN

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D        
from keras.layers import MaxPooling2D       
from keras.layers import Flatten            
from keras.layers import Dense 
from keras.layers import Dropout

from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.preprocessing import image
import numpy as np

### **Parte 2:** Construcción de la CNN

In [3]:
#Se crea y se inicializa la CNN
classifier = Sequential()

#Paso 1 - Convolución------------------------------------
classifier.add(Convolution2D(
    filters=32,
    kernel_size=(3,3),
    input_shape = (64,64,3),
    activation="relu"
))

#Paso 2 - Max Pooling------------------------------------
classifier.add(MaxPooling2D(pool_size=(2,2)))

#Se agrega una segunda capa de convolución y de max pooling para mejorar la red
classifier.add(Convolution2D(
    filters=64,
    kernel_size=(3,3),
    activation="relu"
))

classifier.add(MaxPooling2D(pool_size=(2,2)))

#Se agrega una tercer capa de convolución y de max pooling para mejorar la red
classifier.add(Convolution2D(
    filters=128,
    kernel_size=(3,3),
    activation="relu"
))

classifier.add(MaxPooling2D(pool_size=(2,2)))

#Paso 3 - Flattening-------------------------------------
classifier.add(Flatten())

#Paso 4 - Full Connection--------------------------------
classifier.add(Dense(
    units=64,
    activation="relu"
))

classifier.add(Dropout(rate=0.5))

classifier.add(Dense(#Capa de salida
    units=1,
    activation="sigmoid"
))

#Compilación de la CNN
classifier.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

### **Parte 3:** Ajustar la imágenes para entrenar la CNN

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

training_dataset = train_datagen.flow_from_directory(
    directory='dataset/training_set/',
    target_size=(64, 64),
    batch_size=40,
    class_mode='binary'
)

testing_dataset = test_datagen.flow_from_directory(
    directory='dataset/test_set/',
    target_size=(64, 64),
    batch_size=40,
    class_mode='binary'
)

classifier.fit_generator(
    training_dataset,
    steps_per_epoch=200,
    epochs=25,
    validation_data=testing_dataset,
    validation_steps=50
)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
200/200 [==============================] - 177s 879ms/step - loss: 0.6935 - accuracy: 0.5063 - val_loss: 0.6923 - val_accuracy: 0.5010
Epoch 2/25
200/200 [==============================] - 98s 488ms/step - loss: 0.6846 - accuracy: 0.5491 - val_loss: 0.6750 - val_accuracy: 0.5995
Epoch 3/25
200/200 [==============================] - 99s 494ms/step - loss: 0.6645 - accuracy: 0.5940 - val_loss: 0.6398 - val_accuracy: 0.6325
Epoch 4/25
200/200 [==============================] - 103s 513ms/step - loss: 0.6223 - accuracy: 0.6594 - val_loss: 0.5933 - val_accuracy: 0.6860
Epoch 5/25
200/200 [==============================] - 93s 463ms/step - loss: 0.5849 - accuracy: 0.7024 - val_loss: 0.5396 - val_accuracy: 0.7300
Epoch 6/25
200/200 [==============================] - 119s 596ms/step - loss: 0.5434 - accuracy: 0.7372 - val_loss: 0.5123 - val_accuracy: 0.7465
Epoch 7/25
200/200 [=======================

### **Parte 4:** Realizar una simple predicción

In [5]:
#Carga la imagen de prueba
imagen_prueba = image.load_img(
    'dataset/single_prediction/cat_or_dog_1.jpg',
    target_size = (64, 64)
)

#Convierte la imagen a un array numpy
imagen_array = image.img_to_array(imagen_prueba)
imagen_array = np.expand_dims(imagen_array, axis=0)

# Normalizar la imagen
imagen_array /= 255.0

#Realizar la predicción
prediccion = classifier.predict(imagen_array)

1/1 [==============================] - 1s 576ms/step


A continuación, se procede a verificar cómo están etiquetadas las clases.

In [6]:
training_dataset.class_indices

{'cats': 0, 'dogs': 1}

* Si la probabilidad predicha es cercana a 1, el modelo está prediciendo con alta confianza la clase etiquetada como 1 ('dogs').
* Si la probabilidad predicha es cercana a 0, el modelo está prediciendo con alta confianza la clase etiquetada como 0 ('cats').

Verificando la predicción

In [7]:
prediccion[0][0]

0.9999703

**Conclusión:** Dado que el resultado de la predicción es 0.99999756 y la clase 'dogs' ('perros') 
está etiquetada como 1, puedes interpretar esa predicción como altamente indicativa 
de que la imagen pertenece a la clase 'dogs' ('perros'). En este caso, una probabilidad 
cercana a 1 sugiere una alta confianza del modelo en que la imagen es un perro según la 
configuración de etiquetas y clases.